<a href="https://colab.research.google.com/github/Subhash11520/PRODIGY_GA_05/blob/main/Neural_Style_Transfer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Step 1: Install required packages
!pip install ipywidgets matplotlib tensorflow --quiet


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 3.6 MB/s eta 0:00:00


In [1]:
# Step 2: Imports and widget setup
import io
import IPython.display
import ipywidgets as widgets
from IPython.display import display
import numpy as np
import tensorflow as tf
import PIL.Image

output_box = widgets.Output()

# Global file paths
content_path = "content_img.jpg"
style_path = "style_img.jpg"
stylized_path = "stylized_output.jpg"


In [2]:
# Step 3: Load image function with resizing
def load_img(path_to_img, max_dim=512):
    img = tf.io.read_file(path_to_img)
    img = tf.image.decode_image(img, channels=3)
    img = tf.image.convert_image_dtype(img, tf.float32)

    shape = tf.cast(tf.shape(img)[:-1], tf.float32)
    long_dim = max(shape)
    scale = max_dim / long_dim

    new_shape = tf.cast(shape * scale, tf.int32)
    img = tf.image.resize(img, new_shape)
    img = img[tf.newaxis, :]
    return img


In [3]:
# Step 4: TensorFlow hub model for style transfer
import tensorflow_hub as hub
hub_model = hub.load('https://tfhub.dev/google/magenta/arbitrary-image-stylization-v1-256/2')


In [4]:
# Step 5: Style transfer function
def run_style_transfer(content_path, style_path, max_dim):
    content_image = load_img(content_path, max_dim)
    style_image = load_img(style_path, max_dim)
    stylized_image = hub_model(tf.constant(content_image), tf.constant(style_image))[0]

    # Convert tensor to image and save
    tensor = tf.squeeze(stylized_image, axis=0)
    image = tf.image.convert_image_dtype(tensor, dtype=tf.uint8)
    encoded = tf.image.encode_jpeg(image)

    tf.io.write_file(stylized_path, encoded)

    return content_image, style_image, stylized_image


In [5]:
# Step 6: Image display helper
def tensor_to_pil(tensor):
    tensor = tf.squeeze(tensor, axis=0)
    tensor = tf.clip_by_value(tensor, 0.0, 1.0)
    return PIL.Image.fromarray(np.uint8(tensor.numpy() * 255))

def show_images(content, style, stylized):
    content_img = tensor_to_pil(content)
    style_img = tensor_to_pil(style)
    stylized_img = tensor_to_pil(stylized)

    display(widgets.HBox([
        widgets.VBox([widgets.Label("Content Image"), widgets.Image(value=image_to_bytes(content_img), format='jpg')]),
        widgets.VBox([widgets.Label("Style Image"), widgets.Image(value=image_to_bytes(style_img), format='jpg')]),
        widgets.VBox([widgets.Label("Stylized Image"), widgets.Image(value=image_to_bytes(stylized_img), format='jpg')])
    ]))

def image_to_bytes(img):
    with io.BytesIO() as output:
        img.save(output, format="JPEG")
        return output.getvalue()


In [6]:
# Step 7: Upload widgets and slider
upload_content = widgets.FileUpload(accept='image/*', multiple=False)
upload_style = widgets.FileUpload(accept='image/*', multiple=False)
apply_button = widgets.Button(description="Apply Style Transfer", button_style='success')
resolution_slider = widgets.IntSlider(value=512, min=128, max=1024, step=64, description='Max Res:')
progress_bar = widgets.FloatProgress(value=0.0, min=0.0, max=1.0, description='Progress:', bar_style='info')

def save_uploaded_file(upload_widget, path):
    if upload_widget.value:
        file_info = list(upload_widget.value.values())[0]
        with open(path, 'wb') as f:
            f.write(file_info['content'])

def on_button_clicked(b):
    with output_box:
        output_box.clear_output()
        print("Running style transfer...")
        progress_bar.value = 0.2

        save_uploaded_file(upload_content, content_path)
        save_uploaded_file(upload_style, style_path)

        progress_bar.value = 0.5

        content, style, result = run_style_transfer(content_path, style_path, resolution_slider.value)
        progress_bar.value = 0.8

        show_images(content, style, result)
        progress_bar.value = 1.0

        display(widgets.HTML(
            f'<b style="color:lightblue">Stylized image (max dimension: {resolution_slider.value}px) is ready for download.</b><br>'
            f'<a href="/files/{stylized_path}" target="_blank" download>📥 Download stylized_output.jpg</a>'
        ))

apply_button.on_click(on_button_clicked)


In [7]:
# Step 8: Show the full UI
display(widgets.HTML("<h2>Neural Style Transfer UI</h2>"))
display(widgets.HBox([upload_content, upload_style]))
display(resolution_slider)
display(apply_button)
display(progress_bar)
display(output_box)


HTML(value='<h2>Neural Style Transfer UI</h2>')

IntSlider(value=512, description='Max Res:', max=1024, min=128, step=64)

Button(button_style='success', description='Apply Style Transfer', style=ButtonStyle())

FloatProgress(value=0.0, bar_style='info', description='Progress:', max=1.0)

Output()